<h1>Búsqueda No Informada y Heurística</h1>
<p>
    En general, un problema de búsqueda se compone de:
    <ul>
        <li>Un nodo inicial</li>
        <li>Una función que devuelve los hijos del nodo</li>
        <li>Una función que booleana que devuelve TRUE si el nodo es uno de los objetivos</li>
        <li>Una función que devuelve la heurística del nodo</li>
    </ul>
</p>
La siguiente función provee una clase abstracta para un problema de búsqueda

In [3]:
class generic_search_problem(object):
    
    def is_goal(self,node,goal):
        raise NotImplementedError("is_goal")
    
    def get_children(self,node):
        raise NotImplementedError("get_children")
    
    def search(self):
        raise NotImplementedError("search")


In [4]:
from heapq import heappush,heappop
import numpy as np

class puzzle(object):
    
    def __init__(self,node,level):
        self.state=node
        self.level=level
        
    def __hash__(self):
        return hash(np.array_str(self.state.ravel()))
        
    def __eq__(self,other):
        return np.array_equal(other.state,self.state)

    
    def __gt__(self,other):
        return self.level>other.level

    def __lt__(self,other):
        return self.level<other.level
   
    def pretty_print(self):
        return np.array_str(self.state.ravel())
 
    def solvable(self):
        return len([(a,b) for i,a in enumerate(self.state.ravel()) for b in self.state.ravel()[i:] if a>b & b!=0])%2==0
    
    def get_children(self):
        child_list=set()
        dim=self.state.shape[0]
        i,j=map(int,np.where(self.state==0))
        #print i,j
        if (j>0):
            child=self.state.copy()
            child[i,j]=self.state[i,j-1]
            child[i,j-1]=0
            p=puzzle(child,self.level+1)
            child_list.add(p)
        if (j<dim-1):
            child=self.state.copy()
            child[i,j]=self.state[i,j+1]
            child[i,j+1]=0
            p=puzzle(child,self.level+1)
            child_list.add(p)
        if (i>0):
            child=self.state.copy()
            child[i,j]=self.state[i-1,j]
            child[i-1,j]=0
            p=puzzle(child,self.level+1)
            child_list.add(p)
        if (i<dim-1):
            child=self.state.copy()
            child[i,j]=self.state[i+1,j]
            child[i+1,j]=0
            p=puzzle(child,self.level+1)
            child_list.add(p)
        return child_list

In [13]:
dim=3

goal=puzzle(np.insert(np.arange(1,dim*dim),dim*dim-1,0).reshape(dim,dim),0)

easy_start=puzzle(np.insert(np.arange(1,dim*dim),6,0).reshape(dim,dim),0)



In [14]:
easy_start.state

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

In [12]:
easy_start.solvable()

False

In [15]:
print('nodo objetivo : ')
print(goal.state)
print('nodo inicial : ')
print(easy_start.state)
print('hijos : ')
for p in easy_start.get_children():
    print('Estado padre, nivel : {0}'.format(p.level))
    print(p.state)
    for c in p.get_children():
        print('Estado hijo, nivel : {0}'.format(c.level))
        print(c.state)

nodo objetivo : 
[[1 2 3]
 [4 5 6]
 [7 8 0]]
nodo inicial : 
[[1 2 3]
 [4 5 6]
 [0 7 8]]
hijos : 
Estado padre, nivel : 1
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Estado hijo, nivel : 2
[[1 2 3]
 [4 5 6]
 [0 7 8]]
Estado hijo, nivel : 2
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Estado hijo, nivel : 2
[[1 2 3]
 [4 0 6]
 [7 5 8]]
Estado padre, nivel : 1
[[1 2 3]
 [0 5 6]
 [4 7 8]]
Estado hijo, nivel : 2
[[1 2 3]
 [5 0 6]
 [4 7 8]]
Estado hijo, nivel : 2
[[1 2 3]
 [4 5 6]
 [0 7 8]]
Estado hijo, nivel : 2
[[0 2 3]
 [1 5 6]
 [4 7 8]]


In [19]:
from collections import deque

cola=deque()
cola.append('uno')
cola.append('dos')
cola.append('tres')

In [21]:
cola.pop()

'dos'

In [23]:
from collections import deque

class bfs_puzzle(generic_search_problem):
    
    def __init__(self,start):
        self.state=start
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # queue
        frontier.append(self.state)
        while frontier:
            node=frontier.popleft() # FIFO
            if self.is_goal(node,goal):
                return parents
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited and child not in frontier:
                    parents.update({child:node})
                    frontier.append(child)
        return parents
    
class dfs_puzzle(generic_search_problem):
    
    def __init__(self,start):
        self.state=start
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # stack
        frontier.append(self.state)
        while frontier:
            node=frontier.pop() # LIFO
            if self.is_goal(node,goal):
                return parents
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited and child not in frontier:
                    parents.update({child:node})
                    frontier.append(child)
        return parents
    
def get_path(search_results,goal):
    path=[key  for (key, value) in search_results.items() if key == goal]
    node=goal
    while search_results[node] is not None:
        parent=search_results[node]
        path.append(parent)
        node=parent
    return path

In [24]:
import sys
bfs_tree=bfs_puzzle(easy_start).search(goal)

for p in get_path(bfs_tree,goal):
    print('-------------------------')
    print('nivel : {0}'.format(p.level))
    print(p.state)

print('BFS memoria : {0:.2f} [kB]'.format(sys.getsizeof(bfs_tree)/1024))    

-------------------------
nivel : 2
[[1 2 3]
 [4 5 6]
 [7 8 0]]
-------------------------
nivel : 1
[[1 2 3]
 [4 5 6]
 [7 0 8]]
-------------------------
nivel : 0
[[1 2 3]
 [4 5 6]
 [0 7 8]]
BFS memoria : 0.35 [kB]


In [ ]:
import sys
dfs_tree=dfs_puzzle(easy_start).search(goal)

for p in get_path(dfs_tree,goal):
    print('-------------------------')
    print('nivel : {0}'.format(p.level))
    print(p.state)

print('DFS memoria : {0:.2f} [kB]'.format(sys.getsizeof(dfs_tree)/1024))

In [28]:
easy_start.state

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

In [30]:
class ids_puzzle(generic_search_problem):
    
    def __init__(self,start):
        self.state=start
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal,bound):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # stack
        frontier.append(self.state)
        while frontier:
            node=frontier.pop() # LIFO
            if self.is_goal(node,goal):
                return parents
            visited.add(node)
            if node.level<bound:
                for child in self.get_children(node):
                    if child not in visited and child not in frontier:
                        parents.update({child:node})
                        frontier.append(child)
        return parents

In [34]:
import sys
c=2
ids_tree=ids_puzzle(easy_start).search(goal,bound=c)

#for p in get_path(ids_tree,goal):
#    print('-------------------------')
#    print('nivel : {0}'.format(p.level))
#    print(p.state)

print('cota :{0}, memoria : {1:.2f} [kB]'.format(c,sys.getsizeof(ids_tree)/1024))  

cota :2, memoria : 0.35 [kB]


In [35]:
goal in ids_tree

True

In [9]:
for p in ids_tree:
  print(p.pretty_print())

[1 2 3 4 5 6 0 7 8]
[1 2 3 0 5 6 4 7 8]
[1 2 3 4 5 6 7 0 8]


In [36]:
while True:
    random_start=puzzle(np.random.permutation(np.arange(9)).reshape(dim,dim),0)
    if random_start.solvable() : break

In [37]:
random_start.state

array([[6, 2, 7],
       [0, 8, 3],
       [4, 1, 5]])

In [38]:
bfs_tree=bfs_puzzle(random_start).search(goal)

for p in get_path(bfs_tree,goal):
    print('-------------------------')
    print('nivel : {0}'.format(p.level))
    print(p.state)

print('BFS memoria : {0:.2f} [kB]'.format(sys.getsizeof(bfs_tree)/1024))

KeyboardInterrupt: 

In [ ]:
dfs_tree=dfs_puzzle(random_start).search(goal)

for p in get_path(dfs_tree,goal):
    print('-------------------------')
    print('nivel : {0}'.format(p.level))
    print(p.state)

print('BFS memoria : {0:.2f} [kB]'.format(sys.getsizeof(dfs_tree)/1024))

In [39]:
import sys
ids_tree={}
c=1
while goal not in ids_tree:
    ids_tree=ids_puzzle(random_start).search(goal,bound=c)
    c=c+1
    print('cota :{0}, memoria : {1:.2f} [kB]'.format(c,sys.getsizeof(ids_tree)/1024))   

cota :2, memoria : 0.23 [kB]
cota :3, memoria : 0.35 [kB]
cota :4, memoria : 0.62 [kB]
cota :5, memoria : 1.15 [kB]
cota :6, memoria : 2.22 [kB]
cota :7, memoria : 4.59 [kB]
cota :8, memoria : 9.09 [kB]
cota :9, memoria : 9.09 [kB]
cota :10, memoria : 18.09 [kB]
cota :11, memoria : 18.09 [kB]
cota :12, memoria : 36.09 [kB]
cota :13, memoria : 36.09 [kB]
cota :14, memoria : 72.09 [kB]
cota :15, memoria : 72.09 [kB]
cota :16, memoria : 144.09 [kB]
cota :17, memoria : 144.09 [kB]
cota :18, memoria : 288.09 [kB]
cota :19, memoria : 288.09 [kB]
cota :20, memoria : 288.09 [kB]
cota :21, memoria : 288.09 [kB]
cota :22, memoria : 576.09 [kB]
cota :23, memoria : 288.09 [kB]
cota :24, memoria : 576.09 [kB]
cota :25, memoria : 576.09 [kB]
cota :26, memoria : 1280.09 [kB]
cota :27, memoria : 1280.09 [kB]
cota :28, memoria : 2560.09 [kB]
cota :29, memoria : 576.09 [kB]
cota :30, memoria : 2560.09 [kB]
cota :31, memoria : 1280.09 [kB]
cota :32, memoria : 2560.09 [kB]
cota :33, memoria : 2560.09 [kB]

In [41]:
len(ids_tree)

32460

<h2>Búsqueda Heurística</h2>
A diferencia de la búqueda no informada, la estrategia de búsqueda denominada best-first es un método del tipo greedy que incorpora al costo $h(n)$ del nodo $n$ hacia el objetivo. La estrategia de búsqueda greedy best-first selecciona el candidato $n \in \operatorname{GET\_CHILDREN}$ con
mejor puntuación $f(n) = h(n)$ en cada momento.

la estrategia de búsqueda A* es también un método del tipo greedy que incorpora al costo $h(n)$ del nodo $n$ hacia el objetivo así como también el costo acumulado $g(n)$ de llegar hacia $n$. La estrategia de búsqueda greedy best-first selecciona el candidato $n \in \operatorname{GET\_CHILDREN}$ con
mejor puntuación $f(n) = h(n)+g(n)$ en cada momento.

<h1>Implementación A* Puzzle 8</h1>
<p>
El problema del 8-puzzle fue inventado en 1870 y se juega en una grilla de 3x3 con 9 bloques cuadrados. Cada bloque tiene una etiqueta con un número del 1 al 8 y existe un espacio en blanco en el cual el jugador puede mover un bloque adyacente.   
</p>
    

In [ ]:
from IPython.display import Image
#Image(filename='../figures/8_puzzle.png') 
    

In [43]:
import heapq

pq=list()
heappush(pq,(3,'uno'))
heappush(pq,(4,'dos'))
heappush(pq,(5,'uno'))

In [45]:
heappop(pq)

(4, 'dos')

<p>
Cada estado del puzzle puede ser visto como un vértice de un grafo tipo árbol, cuyo nodo raiz es el estado inicial del puzzle. Las estrategias de búsqueda no-informada no utilizan ningún criterio para ponderar las soluciones y por ende dependen en gran medida de la complejidad del problema.</p>
<p>
<ul>
<li>
De modo de diseñar algoritmos ''inteligentes'', es necesario reducir el espacio de b{usqueda y concentrarse en los estados que sean  atractivos.
</li>
<li>
Dado la alta complejidad de los problemas, no es posible recorrer todos los caminos y por lo tanto solo podemos evaluar estados que sean potencialmente atractivos.
<li>
La estrategia de búsqueda A* utiliza una combinación de búsqueda local:
</li>

\begin{align}
f(n)=g(n)+h(n)
\end{align}

Donde $g(n)$ corresponde al costo de la ruta desde el nodo inicial al nodo $n$ y $h(n)$ es el costo estimado (cota inferior) desde $n$ hasta el nodo objetivo.
</ul>

Para el problema del puzzle, utlizamos las siguientes heurísticas:

<ul>
<li>
Distancia Hamming : Número de bloques en la posición incorrecta.
</li>
<li>
Distancia Manhattan : Suma de las distancias (suma de la distancia horizontal y vertical) desde el bloque hasta la posición objetivo ($d_M=\vert X_n-X_g \vert + \vert Y_n-Y_g \vert$).
</li>


In [ ]:
from IPython.display import Image
#Image(filename='../figures/8_puzzle_distancias.png') 

In [46]:
def manhattan(node,goal):
    dim=node.state.shape[0]
    diff_mat=np.zeros((dim,dim))
    for i in range(dim):
        for j in range(dim):
            if goal.state[i,j]!=0:
                u,v=map(np.int,np.where(node.state==goal.state[i,j]))
                diff_mat[i,j]=abs(i-u)+abs(j-v)
    return int(diff_mat.sum())

def hamming(node,goal):
    dim=node.state.shape[0]
    diff_mat=np.zeros((dim,dim))
    diff_mat=node.state!=goal.state
    dist=np.sum(diff_mat.astype(int).ravel())
    if dist>0:
        dist=dist-1
    return dist
    


In [47]:
easy_start=puzzle(np.insert(np.arange(1,dim*dim),5,0).reshape(dim,dim),0)
print(easy_start.state)
print(goal.state)
print('Distancia hamming : {0}'.format(hamming(easy_start,goal)))
print('Distancia manhattan : {0}'.format(manhattan(easy_start,goal)))

[[1 2 3]
 [4 5 0]
 [6 7 8]]
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Distancia hamming : 3
Distancia manhattan : 5


/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


In [48]:
class bestfirst_puzzle(generic_search_problem):
    
    def __init__(self,start,heuristic):
        self.state=start
        self.heuristic=heuristic
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=[]
        fun=self.heuristic(self.state,goal)
        heappush(frontier,(fun,self.state))
        while frontier:
            node=heappop(frontier)[1]
            if self.is_goal(node,goal):
                return parents,len(frontier),node.level
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited:
                    parents.update({child:node})
                    fun=self.heuristic(child,goal)
                    heappush(frontier,(fun,child))
        return parents,len(frontier),node.level

In [49]:
random_start.pretty_print()

'[6 2 7 0 8 3 4 1 5]'

In [50]:
solver=bestfirst_puzzle(random_start,hamming)
bf_tree,k,h=solver.search(goal)
path=get_path(bf_tree,goal)

print(bf_tree[goal].level)
#for p in get_path(bf_tree,goal):
#    print('-------------------------')
#    print('nivel : {0}'.format(p.level))
#    print(p.state)

print('Best-First memoria (Hamming): {0:.2f} [kB]'.format(sys.getsizeof(bf_tree)/1024))  
    

50
Best-First memoria (Hamming): 72.09 [kB]


In [51]:
len(bf_tree)

1791

In [53]:
solver=bestfirst_puzzle(random_start,manhattan)
bf_tree,k,h=solver.search(goal)
path=get_path(bf_tree,goal)

print(bf_tree[goal].level)
print('Best-First memoria (Manhattan): {0:.2f} [kB]'.format(sys.getsizeof(bf_tree)/1024)) 

52
Best-First memoria (Manhattan): 18.09 [kB]


/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


In [56]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

hamming_path=[]
hamming_frontier=[]
manhattan_path=[]
manhattan_frontier=[]

        
for i in range(100):
    while True:
        random_start=puzzle(np.random.permutation(np.arange(dim*dim)).reshape((dim,dim)),0)
        if random_start.solvable() : break
    solver1=bestfirst_puzzle(random_start,hamming)
    solver2=bestfirst_puzzle(random_start,manhattan)
    p1,i,j=solver1.search(goal)
    p2,k,h=solver2.search(goal)
    hamming_path.append(j)
    hamming_frontier.append(i)
    manhattan_path.append(h)
    manhattan_frontier.append(k)
    

plt.figure(figsize=(20,15))
plt.plot([1,2,3])
plt.subplot(221)
plt.hist(hamming_path,bins=20 , histtype='stepfilled', color='b', alpha=0.5, label='Hamming')
plt.hist(manhattan_path,bins=20, histtype='stepfilled',  color='r', alpha=0.5, label='Manhattan') 
plt.title("Path length Best-First")
plt.legend()
plt.subplot(222)
plt.hist(hamming_frontier,bins=20, histtype='stepfilled',  color='b', alpha=0.5, label='Hamming') 
plt.hist(manhattan_frontier,bins=20, histtype='stepfilled', color='r', alpha=0.5, label='Manhattan') 
plt.title("Frontier length Best-First")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [58]:
class astar_puzzle(generic_search_problem):
    
    def __init__(self,start,heuristic):
        self.state=start
        self.heuristic=heuristic
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=[]
        fun=self.heuristic(self.state,goal)+self.state.level
        heappush(frontier,(fun,self.state))
        while frontier:
            node=heappop(frontier)[1]
            if self.is_goal(node,goal):
                return parents,len(frontier),node.level
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited:
                    parents.update({child:node})
                    fun=self.heuristic(child,goal)+child.level
                    heappush(frontier,(fun,child))
        return parents,len(frontier),node.level

In [59]:
astar_solver=astar_puzzle(random_start,hamming)
astar_tree,k,h=astar_solver.search(goal)
path=get_path(astar_tree,goal)

#for p in path:
#    print(p.state,p.level)
#    print('---------------')
#print('frontier : {0}, visited : {1}'.format(k,h))
print('frontier : {0}, visited : {1}'.format(k,h))
print(astar_tree[goal].level)    
print('A-Star Hamming memoria : {0:.2f} [kB]'.format(sys.getsizeof(astar_tree)/1024)) 

frontier : 9340, visited : 23
22
A-Star Hamming memoria : 576.09 [kB]


In [60]:
astar_solver=astar_puzzle(random_start,manhattan)
astar_tree,k,h=astar_solver.search(goal)
path=get_path(astar_tree,goal)

#for p in path:
#    print(p.state,p.level)
#    print('---------------')
print('frontier : {0}, visited : {1}'.format(k,h))
print(astar_tree[goal].level)    
print('A-Star Manhattan memoria : {0:.2f} [kB]'.format(sys.getsizeof(astar_tree)/1024)) 

frontier : 701, visited : 23
22
A-Star Manhattan memoria : 72.09 [kB]


/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


In [61]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

hamming_path=[]
hamming_frontier=[]
manhattan_path=[]
manhattan_frontier=[]

        
for i in range(100):
    while True:
        random_start=puzzle(np.random.permutation(np.arange(dim*dim)).reshape((dim,dim)),0)
        if random_start.solvable() : break
    solver1=astar_puzzle(random_start,hamming)
    solver2=astar_puzzle(random_start,manhattan)
    p1,i,j=solver1.search(goal)
    p2,k,h=solver2.search(goal)
    hamming_path.append(j)
    hamming_frontier.append(i)
    manhattan_path.append(h)
    manhattan_frontier.append(k)
    

plt.figure(figsize=(20,15))
plt.plot([1,2,3])
plt.subplot(221)
plt.hist(hamming_path,bins=20 , histtype='stepfilled', color='b', alpha=0.5, label='Hamming')
plt.hist(manhattan_path,bins=20, histtype='stepfilled',  color='r', alpha=0.5, label='Manhattan') 
plt.title("Path length A star")
plt.legend()
plt.subplot(222)
plt.hist(hamming_frontier,bins=20, histtype='stepfilled',  color='b', alpha=0.5, label='Hamming') 
plt.hist(manhattan_frontier,bins=20, histtype='stepfilled', color='r', alpha=0.5, label='Manhattan') 
plt.title("Frontier length A star")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [62]:
def gen_state(level=1):
    node=goal
    visited={goal}
    c=0
    while c<level:
        nodes=[p for p in node.get_children() if p not in visited]
        ind=np.random.randint(len(nodes),size=1)
        node=nodes[np.random.randint(len(nodes),size=1)[0]]
        visited.add(node)
        c=c+1
    return node

In [64]:
import time
new_node=gen_state(level=100)

solver=bestfirst_puzzle(new_node,hamming)
t1=time.time()
bf_tree,k1,d1=solver.search(goal)
print('Tiempo Best-First: ',time.time()-t1)
print('Best-First memoria : {0:.2f} [kB], Nivel : '.format(sys.getsizeof(bf_tree)/1024),d1)
print('----------------------------------')
solver=astar_puzzle(new_node,hamming)
t2=time.time()
astar_tree,k2,d2=solver.search(goal)
print('Tiempo A-star: ',time.time()-t2)
print('A-star memoria : {0:.2f} [kB], Nivel : '.format(sys.getsizeof(astar_tree)/1024),d2)

Tiempo Best-First:  0.07938098907470703
Best-First memoria : 18.09 [kB], Nivel :  154
----------------------------------
Tiempo A-star:  1.5561439990997314
A-star memoria : 576.09 [kB], Nivel :  122


In [ ]:
solver=bestfirst_puzzle(new_node,hamming)
t1=time.time()
bf_tree,k1,d1=solver.search(goal)
print('Tiempo Best-First: ',time.time()-t1)
print('Best-First memoria : {0:.2f} [kB], Nivel : '.format(sys.getsizeof(bf_tree)/1024),d1)
print('----------------------------------')
solver=astar_puzzle(new_node,hamming)
t2=time.time()
astar_tree,k2,d2=solver.search(goal)
print('Tiempo A-star: ',time.time()-t2)
print('A-star memoria : {0:.2f} [kB], Nivel : '.format(sys.getsizeof(astar_tree)/1024),d2)

Tiempo Best-First:  0.45841526985168457
Best-First memoria : 36.10 [kB], Nivel :  158
----------------------------------
Tiempo A-star:  1.0276892185211182
A-star memoria : 72.09 [kB], Nivel :  118


In [65]:
class ida_puzzle(generic_search_problem):
    
    def __init__(self,start,heuristic):
        self.state=start
        self.heuristic=heuristic
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal,bound):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # stack
        frontier.append(self.state)
        while frontier:
            max_val=np.inf
            node=frontier.pop() # LIFO
            if self.is_goal(node,goal):
                return parents,max_val
            visited.add(node)
            fun=node.level+self.heuristic(node,goal)
            for child in self.get_children(node):
              nex_val=child.level+self.heuristic(child,goal)
              print('cota padre : {0}, cota hijo : {1}'.format(fun,nex_val))
              if nex_val<=bound and child not in visited:
                parents.update({child:node})
                frontier.append(child)
              else:
                max_val=min(max_val,nex_val)
        return parents,max_val

In [66]:
import sys
ida_tree={}
solver=ida_puzzle(random_start,manhattan)
max_val=solver.heuristic(random_start,goal)


/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


In [67]:
ida_tree,max_val=solver.search(goal,bound=max_val)

cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17


/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


In [73]:
max_val

17

In [68]:
while goal not in ida_tree:
    ida_tree,max_val=solver.search(goal,bound=max_val)
    print('cota :{0}, memoria : {1:.2f} [kB]'.format(max_val,sys.getsizeof(ida_tree)/1024))  

/var/folders/wd/0sm_658d23ldnfyfjn6y4s6m0000gn/T/ipykernel_4795/701123217.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  u,v=map(np.int,np.where(node.state==goal.state[i,j]))


cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 19
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 15
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 17
cota padre : 17, cota hijo : 19
cota padre : 17, cota hijo : 19
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota padre : 15, cota hijo : 17
cota pad

cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 19, cota hijo : 21
cota padre : 19, cota hijo : 19
cota padre : 19, cota hijo : 21
cota padre : 21, cota hijo : 23
cota padre : 21, cota hijo : 21
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 23
cota padre : 23, cota hijo : 25
cota padre : 19, cota hijo : 21
cota padre : 19, cota hijo : 21
cota padre : 21, cota hijo : 23
cota padre : 21, cota hijo : 21
cota padre : 21, cota hijo : 23
cota padre : 23, cota hijo : 23
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 23
cota padre : 23, cota hijo : 23
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 23
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 23, cota hijo : 25
cota padre : 21, cota hijo : 21
cota padre : 21, cota hijo : 23
cota pad